In [1]:
# Michael Wasserstein
# Plot_Terrain_Cross_Sections.ipynb
# 10/17/2024
# Notebook plots a north-south or east-west cross section of terrain along the wasatch,
# Using widgits.

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.cm import ScalarMappable
import matplotlib.patheffects as PathEffects
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib
from cartopy import crs
from cartopy.feature import NaturalEarthFeature, COLORS
from netCDF4 import Dataset
from wrf import (getvar, to_np, get_cartopy, latlon_coords, vertcross,
                 cartopy_xlim, cartopy_ylim, interpline, CoordPair)
import wrf
import glob
import os, sys
import pandas as pd
import datetime
import pyart
import xarray as xr

import ipywidgets as widgets
from ipywidgets import interact

######## User input arguments #############
# import argparse
# parser = argparse.ArgumentParser()

# parser.add_argument("-r", "--run", help="WRF run of interest")
# parser.add_argument("-p", "--path", help="Wrf path - where is data (1 or 2")

# args = parser.parse_args()

# Get user inputs
# run = str(args.run)
# path = int(args.path)
run = '13'
path = 12

run_number = '{}'.format(run).zfill(2)

# Base paths
base = '/uufs/chpc.utah.edu/common/home/'
home = base + 'u1371671/'
# paths for data
if path ==1:
    base_path = base + 'steenburgh-group12/michael/wrf/'
else:
    base_path = base + 'steenburgh-group12/michael/wrf{}/'.format(path)
WRF_path = base_path + 'wrf_runs/wrf_{}/run/'.format(run_number)
WPS_path = base_path + 'WPS/'

# paths for saving fig
Fig_dir = home + 'WRF/Figures_{}/wrf_{}/Terrain_Testing/'.format(path,run_number)

if os.path.exists(Fig_dir) == False:
    os.mkdir(Fig_dir)
    
# load wrf data
# load in all the wrf output data files
data_files_d04 = glob.glob(WRF_path  + '*wrfout_d04*') # for the outermost domain
data_files_d04.sort()
data_files_d03 = glob.glob(WRF_path  + '*wrfout_d03*') # for the outermost domain
data_files_d03.sort()

wrf_file = Dataset(data_files_d03[1])

wrf_file_d04 = Dataset(data_files_d04[1])

geobounds = wrf.geo_bounds(wrfin=wrf_file_d04)
min_lat = geobounds.bottom_left.lat
left_lon = geobounds.bottom_left.lon
max_lat = geobounds.top_right.lat
right_lon = geobounds.top_right.lon

for ind, f in enumerate(data_files_d03[7:8]):

    #################################################################################
    #############################   Domain 3 #######################################
    #################################################################################
    # Open dataset for domain 3
    wrf_file_d03 = Dataset(f)

    dbz_d03 = getvar(wrf_file_d03, "dbz", timeidx=-1)

    # Get the times
    init_time = wrf_file_d03.SIMULATION_START_DATE
    init_time = datetime.datetime.strptime(init_time, '%Y-%m-%d_%H:%M:%S')
    init_time_str = datetime.datetime.strftime(init_time, '%b %-d, %Y %H:%M UTC')

    valid_time = pd.to_datetime(dbz_d03.Time.values)
    valid_time_str = datetime.datetime.strftime(valid_time, '%b %-d, %Y %H:%M UTC')

    YYYY = valid_time.year
    mm = valid_time.month
    DD = valid_time.day
    HH = valid_time.hour
    MM = valid_time.minute

    # Get the file as a netcdf dataset
    wrf_file_d04 = Dataset(data_files_d04[ind]) 
    #wrf_file_d04 = Dataset(data_files_d04[34])  # use for testing
    
    
     # Get the WRF variables for domain 3
    ht_d03 = getvar(wrf_file_d03, "z", timeidx=-1)
    ter_d03 = getvar(wrf_file_d03, "ter", timeidx=-1)
    dbz_d03 = getvar(wrf_file_d03, "dbz", timeidx=-1)
    dbz_d03.values = xr.where(dbz_d03.values == -30, np.nan, dbz_d03.values)
    Z_d03 = 10**(dbz_d03/10.) # Use linear Z for interpolation # gets rid of the log
    
    # Get the WRF variables for d04
    ter_d04 = getvar(wrf_file_d04, "ter", timeidx=-1)
    ht_d04 = getvar(wrf_file_d04, "z", timeidx=-1)
    dbz_d04 = getvar(wrf_file_d04, "dbz", timeidx=-1)
    dbz_d04.values = xr.where(dbz_d04.values == -30, np.nan, dbz_d04.values)    
    Z_d04 = 10**(dbz_d04/10.) # Use linear Z for interpolation # gets rid of the log


/uufs/chpc.utah.edu/common/home/u1371671/software/pkg/miniconda3/envs/py37/lib/python3.7/site-packages/gribapi/__init__.py:25: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.23.0
  "You are running version {}".format(min_recommended_version_str, __version__)



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/uufs/chpc.utah.edu/common/home/u1371671/software/pkg/miniconda3/envs/py37/lib/python3.7/site-packages/osgeo/__init__.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
# latitudes_to_test = np.arange(40.4,40.75, 0.01)
# lon_start = -111.895
# lon_end = -111.4

# for lat in latitudes_to_test[5:6]:
    
#     # THe start and end points for the cross section
#     cross_start = CoordPair(lat=lat, lon=lon_start)
#     cross_end = CoordPair(lat=lat, lon=lon_end)

#     # Get the terrain heights along the cross section line
#     ter_line_d03 = interpline(ter_d03, wrfin=wrf_file_d03, start_point=cross_start,
#                             end_point=cross_end)
    
#     # Compute the vertical cross-section interpolation.  Also, include the
#     # lat/lon points along the cross-section in the metadata by setting latlon
#     # to True.
#     z_cross_d03 = vertcross(Z_d03, ht_d03, wrfin=wrf_file_d03,
#                         start_point=cross_start,
#                         end_point=cross_end,
#                         latlon=True, meta=True)



#     #################################################################################
#     #############################   Domain 4 #######################################
#     #################################################################################


#     # Filter to include only coordinate pairs with longitude greater than left_lon
#     # This line of code ensures that you wont be taking a cross section with stuff from outside of the domain
#     filtered_coord_pairs = [cp for cp in z_cross_d03.xy_loc.values if cp.lon > left_lon]

#     # Find the coordinate pair with the longitude closest to left_lon among the filtered pairs
#     # Essentially here, you're finding the coordinate pair from the d03 cross section that is closest to the boundary of domain 4
#     closest_coord_pair_min = min(filtered_coord_pairs, key=lambda cp: abs(cp.lon - left_lon))
#     closest_coord_pair_max = min(z_cross_d03.xy_loc.values, key=lambda cp: abs(cp.lon - right_lon)) # Same for right lon


#     # Not totally sure why steps 3 and 4 need to be done, but it was givent to me by CHATGPT. 
#     # Basically, the closest one to the boundary didnt work, so you need to use one more inwards.
#     # Step 3: Find the index of the closest coordinate pair in the original list
#     original_list = list(z_cross_d03.xy_loc.values)  # Convert to list if not already a list
#     index_of_closest = original_list.index(closest_coord_pair_min)

#     # Step 4: Get the next coordinate pair in the list
#     if index_of_closest < len(original_list) - 1:  # Ensure it's not the last element
#         next_coord_pair = original_list[index_of_closest + 1]
#     else:
#         next_coord_pair = None  # Handle the case where there is no next pair




#     # Coordinates for the start and end of the cross section for d04. You are doing this based on the part of d03 that is within d04
#     cross_start_for_d04 = next_coord_pair
#     cross_end_for_d04 = CoordPair(lon=closest_coord_pair_max.lon, lat=closest_coord_pair_max.lat)


#     # Get the WRF variables for d04
#     ter_d04 = getvar(wrf_file_d04, "ter", timeidx=-1)
#     ht_d04 = getvar(wrf_file_d04, "z", timeidx=-1)
#     dbz_d04 = getvar(wrf_file_d04, "dbz", timeidx=-1)
#     dbz_d04.values = xr.where(dbz_d04.values == -30, np.nan, dbz_d04.values)    
#     Z_d04 = 10**(dbz_d04/10.) # Use linear Z for interpolation # gets rid of the log

   
#     # Terrain for d04
#     # Get the terrain heights along the cross section line
#     ter_line_d04 = interpline(ter_d04, wrfin=wrf_file_d04, start_point=cross_start_for_d04,
#                             end_point=cross_end_for_d04)
    
#     # Compute the vertical cross-section interpolation.  Also, include the
#     # lat/lon points along the cross-section in the metadata by setting latlon
#     # to True.
#     z_cross_d04 = vertcross(Z_d04, ht_d04, wrfin=wrf_file_d04,
#                         start_point=cross_start_for_d04,
#                             end_point=cross_end_for_d04,
#                         latlon=True, meta=True)



#     ############################################################################################
    
    
#     # Create the figure
#     fig, (ax ) = plt.subplots(1,1, figsize=(8,6), facecolor = 'white', sharey=True)

#     dbz_map = plt.get_cmap('jet').copy()
#     dbz_map.set_under('none')

#     # Setting the background color of the plot
#     # using set_facecolor() method
#     ax.set_facecolor("gainsboro")


#     ####################################### d03 plot ##########################################
#     #dbz_levels = np.arange(0, 31., 1.0)
#     xs_d03 = np.arange(0, z_cross_d03.shape[-1], 1)
#     ys_d03 = to_np(z_cross_d03.coords["vertical"])

#     # Fill in the mountain area
#     ht_fill = ax.fill_between(xs_d03, 0, to_np(ter_line_d03),
#                                     facecolor="saddlebrown", zorder = 19)


#     # Do all the ticks and limits before you do the d04 plot
#     #Set the x-ticks to use latitude and longitude labels
#     coord_pairs = to_np(z_cross_d03.coords["xy_loc"])
#     x_ticks = np.arange(coord_pairs.shape[0])
#     x_labels = [pair.latlon_str() for pair in to_np(coord_pairs)]

#     # Set the desired number of x ticks below
#     num_ticks = 5
#     thin = int((len(x_ticks) / num_ticks) + .5)
#     ax.set_xticks(x_ticks[::thin])
#     ax.set_xticklabels(x_labels[::thin], rotation=45, fontsize=8)

#     # Set the x-axis and  y-axis labels
#     ax.set_xlabel("Latitude, Longitude", fontsize=12)
#     ax.set_ylabel("Height (m)", fontsize=12)

#     ax.set_ylim(800,5000)
#     ax.set_xlim(xs_d03[0], xs_d03[-1])


#     ####################################### d04 plot ##########################################
#     # Here you are extracting the x coordinates in domain 3 plot of the start and end of the cross section for domain 4
#     start_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_start_for_d04.lat, longitude=cross_start_for_d04.lon).sel(x_y = 'x').values
#     end_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_end_for_d04.lat, longitude=cross_end_for_d04.lon).sel(x_y = 'x').values


#     # Make an array of x coordinates and y coordinates for plotting d03
#     # Not sure why you need to subtract the first x from d03 cross, but it works
#     #if flight_leg in [1,2,3,12,13,14]:
#     xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, end_x_from_d04-z_cross_d03.xy_loc.values[0].x,z_cross_d04.shape[1])
#     # For the legs where the end of the leg is at the same spot as d03, just use the last point from d03
#     #elif flight_leg in [4,5,6,7,8,9,10,11]:
#     xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, xs_d03[-1],z_cross_d04.shape[1])
#     ys_d04 = to_np(z_cross_d03.coords["vertical"])

#     ht_fill = ax.plot(xs_d04, to_np(ter_line_d04),color = 'black', zorder = 20, label = 'd04 terrain')


#     ax.legend(loc = 'upper left')
#     # Add a title
#     ax.set_title(f"Terrain cross section along {round(lat, 2)}°", fontsize = 10, loc = 'center',)
#     ax.set_title(f'WRF{path} Run {run_number}', fontsize = 10, loc = 'left')
    
#     #######################################################################
#     ################### end terrain plot ##################################
#     #######################################################################
    
#     # Now create an inset axis for the map plot using plt.axes
#     inset_ax = fig.add_axes([0.6, 0.55, 0.3, 0.3], projection=crs.PlateCarree())
    
#     # Create the map in the inset
#     inset_ax.contourf(ter_d03.XLONG, ter_d03.XLAT, ter_d03.values, transform=crs.PlateCarree(), cmap='terrain', levels=60)

#     # Plot the cross-section line on the inset map
#     inset_ax.plot([lon_start, lon_end], [lat, lat], color='red', marker='.', linewidth=2, markersize=14, transform=crs.PlateCarree())

#     #plt.savefig(save_path, dpi = 200, bbox_inches = 'tight')
#     plt.show()
#     plt.close()

# Cross section along constant latitude just Wasatch

In [4]:
def plot_cross_section(lat):
    lon_start = -111.895
    lon_end = -111.4
    
    # THe start and end points for the cross section
    cross_start = CoordPair(lat=lat, lon=lon_start)
    cross_end = CoordPair(lat=lat, lon=lon_end)

    # Get the terrain heights along the cross section line
    ter_line_d03 = interpline(ter_d03, wrfin=wrf_file_d03, start_point=cross_start,
                            end_point=cross_end)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d03 = vertcross(Z_d03, ht_d03, wrfin=wrf_file_d03,
                        start_point=cross_start,
                        end_point=cross_end,
                        latlon=True, meta=True)



    #################################################################################
    #############################   Domain 4 #######################################
    #################################################################################


    # Filter to include only coordinate pairs with longitude greater than left_lon
    # This line of code ensures that you wont be taking a cross section with stuff from outside of the domain
    filtered_coord_pairs = [cp for cp in z_cross_d03.xy_loc.values if cp.lon > left_lon]

    # Find the coordinate pair with the longitude closest to left_lon among the filtered pairs
    # Essentially here, you're finding the coordinate pair from the d03 cross section that is closest to the boundary of domain 4
    closest_coord_pair_min = min(filtered_coord_pairs, key=lambda cp: abs(cp.lon - left_lon))
    closest_coord_pair_max = min(z_cross_d03.xy_loc.values, key=lambda cp: abs(cp.lon - right_lon)) # Same for right lon


    # Not totally sure why steps 3 and 4 need to be done, but it was givent to me by CHATGPT. 
    # Basically, the closest one to the boundary didnt work, so you need to use one more inwards.
    # Step 3: Find the index of the closest coordinate pair in the original list
    original_list = list(z_cross_d03.xy_loc.values)  # Convert to list if not already a list
    index_of_closest = original_list.index(closest_coord_pair_min)

    # Step 4: Get the next coordinate pair in the list
    if index_of_closest < len(original_list) - 1:  # Ensure it's not the last element
        next_coord_pair = original_list[index_of_closest + 1]
    else:
        next_coord_pair = None  # Handle the case where there is no next pair


    # Coordinates for the start and end of the cross section for d04. You are doing this based on the part of d03 that is within d04
    cross_start_for_d04 = next_coord_pair
    cross_end_for_d04 = CoordPair(lon=closest_coord_pair_max.lon, lat=closest_coord_pair_max.lat)

   
    # Terrain for d04
    # Get the terrain heights along the cross section line
    ter_line_d04 = interpline(ter_d04, wrfin=wrf_file_d04, start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d04 = vertcross(Z_d04, ht_d04, wrfin=wrf_file_d04,
                        start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04,
                        latlon=True, meta=True)



    ############################################################################################
    
    
    # Create the figure
    fig, (ax ) = plt.subplots(1,1, figsize=(8,6), facecolor = 'white', sharey=True)

    dbz_map = plt.get_cmap('jet').copy()
    dbz_map.set_under('none')

    # Setting the background color of the plot
    # using set_facecolor() method
    ax.set_facecolor("gainsboro")


    ####################################### d03 plot ##########################################
    #dbz_levels = np.arange(0, 31., 1.0)
    xs_d03 = np.arange(0, z_cross_d03.shape[-1], 1)
    ys_d03 = to_np(z_cross_d03.coords["vertical"])

    # Fill in the mountain area
    ht_fill = ax.fill_between(xs_d03, 0, to_np(ter_line_d03),
                                    facecolor="saddlebrown", zorder = 19)


    # Do all the ticks and limits before you do the d04 plot
    #Set the x-ticks to use latitude and longitude labels
    coord_pairs = to_np(z_cross_d03.coords["xy_loc"])
    x_ticks = np.arange(coord_pairs.shape[0])
    x_labels = [pair.latlon_str() for pair in to_np(coord_pairs)]

    # Set the desired number of x ticks below
    num_ticks = 5
    thin = int((len(x_ticks) / num_ticks) + .5)
    ax.set_xticks(x_ticks[::thin])
    ax.set_xticklabels(x_labels[::thin], rotation=45, fontsize=8)

    # Set the x-axis and  y-axis labels
    ax.set_xlabel("Latitude, Longitude", fontsize=12)
    ax.set_ylabel("Height (m)", fontsize=12)

    ax.set_ylim(800,5000)
    ax.set_xlim(xs_d03[0], xs_d03[-1])


    ####################################### d04 plot ##########################################
    # Here you are extracting the x coordinates in domain 3 plot of the start and end of the cross section for domain 4
    start_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_start_for_d04.lat, longitude=cross_start_for_d04.lon).sel(x_y = 'x').values
    end_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_end_for_d04.lat, longitude=cross_end_for_d04.lon).sel(x_y = 'x').values


    # Make an array of x coordinates and y coordinates for plotting d03
    # Not sure why you need to subtract the first x from d03 cross, but it works
    #xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, end_x_from_d04-z_cross_d03.xy_loc.values[0].x,z_cross_d04.shape[1])
    # For the legs where the end of the leg is at the same spot as d03, just use the last point from d03
    xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, xs_d03[-1],z_cross_d04.shape[1])
    ys_d04 = to_np(z_cross_d04.coords["vertical"])

    # D04 terrain line
    ht_fill = ax.plot(xs_d04, to_np(ter_line_d04),color = 'black', zorder = 20, label = 'd04 terrain')

    # Add legend
    ax.legend(loc = 'upper left')
    
    # Add a title
    ax.set_title(f"Terrain cross section along {round(lat, 2)}°", fontsize = 10, loc = 'center',)
    ax.set_title(f'WRF{path} Run {run_number}', fontsize = 10, loc = 'left')
    
    #######################################################################
    ################### end terrain plot ##################################
    #######################################################################
    
    # Now create an inset axis for the map plot using plt.axes
    inset_ax = fig.add_axes([0.61, 0.57, 0.3, 0.3], projection=crs.PlateCarree())
    
    # Create the map in the inset
    inset_ax.contourf(ter_d03.XLONG, ter_d03.XLAT, ter_d03.values, transform=crs.PlateCarree(), cmap='terrain', levels=60)

    # Plot the cross-section line on the inset map
    inset_ax.plot([lon_start, lon_end], [lat, lat], color='red', marker='.', linewidth=2, markersize=14, transform=crs.PlateCarree(),
                 zorder = 2001)
    
    # d04 box
    inset_ax.add_patch(matplotlib.patches.Rectangle((left_lon, min_lat), right_lon-left_lon, max_lat-min_lat,
             fill=None, lw=2, edgecolor='black', zorder=2000,transform = crs.PlateCarree(), alpha = 0.5))

    plt.show()

# Define the slider using ipywidgets
lat_slider = widgets.FloatSlider(value=40.4, min=min_lat+0.01, max=max_lat-0.01, step=0.01, description='Latitude:')

# Use interact to connect the slider to the plot function
interact(plot_cross_section, lat=lat_slider)

/uufs/chpc.utah.edu/common/home/u1371671/software/pkg/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py:13: DeprecationWarning: `ipykernel.pylab.backend_inline` is deprecated, directly use `matplotlib_inline.backend_inline`
  DeprecationWarning,


interactive(children=(FloatSlider(value=40.45367218017578, description='Latitude:', max=40.729971160888674, mi…

<function __main__.plot_cross_section(lat)>

# Cross section along constant latitude Oquirrhs Wasatch

In [5]:
def plot_cross_section(lat):
    lon_start = -112.3
    lon_end = -111.4
    
# THe start and end points for the cross section
    cross_start = CoordPair(lat=lat, lon=lon_start)
    cross_end = CoordPair(lat=lat, lon=lon_end)

    # Get the terrain heights along the cross section line
    ter_line_d03 = interpline(ter_d03, wrfin=wrf_file_d03, start_point=cross_start,
                            end_point=cross_end)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d03 = vertcross(Z_d03, ht_d03, wrfin=wrf_file_d03,
                        start_point=cross_start,
                        end_point=cross_end,
                        latlon=True, meta=True)



    #################################################################################
    #############################   Domain 4 #######################################
    #################################################################################


    # Filter to include only coordinate pairs with longitude greater than left_lon
    # This line of code ensures that you wont be taking a cross section with stuff from outside of the domain
    filtered_coord_pairs = [cp for cp in z_cross_d03.xy_loc.values if cp.lon > left_lon]

    # Find the coordinate pair with the longitude closest to left_lon among the filtered pairs
    # Essentially here, you're finding the coordinate pair from the d03 cross section that is closest to the boundary of domain 4
    closest_coord_pair_min = min(filtered_coord_pairs, key=lambda cp: abs(cp.lon - left_lon))
    closest_coord_pair_max = min(z_cross_d03.xy_loc.values, key=lambda cp: abs(cp.lon - right_lon)) # Same for right lon


    # Not totally sure why steps 3 and 4 need to be done, but it was givent to me by CHATGPT. 
    # Basically, the closest one to the boundary didnt work, so you need to use one more inwards.
    # Step 3: Find the index of the closest coordinate pair in the original list
    original_list = list(z_cross_d03.xy_loc.values)  # Convert to list if not already a list
    index_of_closest = original_list.index(closest_coord_pair_min)

    # Step 4: Get the next coordinate pair in the list
    if index_of_closest < len(original_list) - 1:  # Ensure it's not the last element
        next_coord_pair = original_list[index_of_closest + 1]
    else:
        next_coord_pair = None  # Handle the case where there is no next pair



    # Coordinates for the start and end of the cross section for d04. You are doing this based on the part of d03 that is within d04
    cross_start_for_d04 = next_coord_pair
    cross_end_for_d04 = CoordPair(lon=closest_coord_pair_max.lon, lat=closest_coord_pair_max.lat)

   
    # Terrain for d04
    # Get the terrain heights along the cross section line
    ter_line_d04 = interpline(ter_d04, wrfin=wrf_file_d04, start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d04 = vertcross(Z_d04, ht_d04, wrfin=wrf_file_d04,
                        start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04,
                        latlon=True, meta=True)



    ############################################################################################
    
    
    # Create the figure
    fig, (ax ) = plt.subplots(1,1, figsize=(8,6), facecolor = 'white', sharey=True)

    dbz_map = plt.get_cmap('jet').copy()
    dbz_map.set_under('none')

    # Setting the background color of the plot
    # using set_facecolor() method
    ax.set_facecolor("gainsboro")


    ####################################### d03 plot ##########################################
    #dbz_levels = np.arange(0, 31., 1.0)
    xs_d03 = np.arange(0, z_cross_d03.shape[-1], 1)
    ys_d03 = to_np(z_cross_d03.coords["vertical"])

    # Fill in the mountain area
    ht_fill = ax.fill_between(xs_d03, 0, to_np(ter_line_d03),
                                    facecolor="saddlebrown", zorder = 19)


    # Do all the ticks and limits before you do the d04 plot
    #Set the x-ticks to use latitude and longitude labels
    coord_pairs = to_np(z_cross_d03.coords["xy_loc"])
    x_ticks = np.arange(coord_pairs.shape[0])
    x_labels = [pair.latlon_str() for pair in to_np(coord_pairs)]

    # Set the desired number of x ticks below
    num_ticks = 5
    thin = int((len(x_ticks) / num_ticks) + .5)
    ax.set_xticks(x_ticks[::thin])
    ax.set_xticklabels(x_labels[::thin], rotation=45, fontsize=8)

    # Set the x-axis and  y-axis labels
    ax.set_xlabel("Latitude, Longitude", fontsize=12)
    ax.set_ylabel("Height (m)", fontsize=12)

    ax.set_ylim(800,5000)
    ax.set_xlim(xs_d03[0], xs_d03[-1])


    ####################################### d04 plot ##########################################
    # Here you are extracting the x coordinates in domain 3 plot of the start and end of the cross section for domain 4
    start_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_start_for_d04.lat, longitude=cross_start_for_d04.lon).sel(x_y = 'x').values
    end_x_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_end_for_d04.lat, longitude=cross_end_for_d04.lon).sel(x_y = 'x').values


    # Make an array of x coordinates and y coordinates for plotting d03
    # Not sure why you need to subtract the first x from d03 cross, but it works
    #xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, end_x_from_d04-z_cross_d03.xy_loc.values[0].x,z_cross_d04.shape[1])
    # For the legs where the end of the leg is at the same spot as d03, just use the last point from d03
    xs_d04 = np.linspace(start_x_from_d04-z_cross_d03.xy_loc.values[0].x, xs_d03[-1],z_cross_d04.shape[1])
    ys_d04 = to_np(z_cross_d04.coords["vertical"])

    # D04 terrain line
    ht_fill = ax.plot(xs_d04, to_np(ter_line_d04),color = 'black', zorder = 20, label = 'd04 terrain')

    # Add legend
    ax.legend(loc = 'upper left')
    
    # Add a title
    ax.set_title(f"Terrain cross section along {round(lat, 2)}°", fontsize = 10, loc = 'center',)
    ax.set_title(f'WRF{path} Run {run_number}', fontsize = 10, loc = 'left')
    
    #######################################################################
    ################### end terrain plot ##################################
    #######################################################################
    
    # Now create an inset axis for the map plot using plt.axes
    inset_ax = fig.add_axes([0.61, 0.57, 0.3, 0.3], projection=crs.PlateCarree())
    
    # Create the map in the inset
    inset_ax.contourf(ter_d03.XLONG, ter_d03.XLAT, ter_d03.values, transform=crs.PlateCarree(), cmap='terrain', levels=60)

    # Plot the cross-section line on the inset map
    inset_ax.plot([lon_start, lon_end], [lat, lat], color='red', marker='.', linewidth=2, markersize=14, transform=crs.PlateCarree(),
                 zorder = 2001)
    
    # d04 box
    inset_ax.add_patch(matplotlib.patches.Rectangle((left_lon, min_lat), right_lon-left_lon, max_lat-min_lat,
             fill=None, lw=2, edgecolor='black', zorder=2000,transform = crs.PlateCarree(), alpha = 0.5))

    plt.show()


# Define the slider using ipywidgets
lat_slider = widgets.FloatSlider(value=40.4, min=min_lat+0.01, max=max_lat-0.01, step=0.01, description='Latitude:')

# Use interact to connect the slider to the plot function
interact(plot_cross_section, lat=lat_slider)

interactive(children=(FloatSlider(value=40.45367218017578, description='Latitude:', max=40.729971160888674, mi…

<function __main__.plot_cross_section(lat)>

# Cross section along constant longitude

In [6]:
def plot_cross_section(lon):
    lat_start = 40.3
    lat_end = 40.8
    
# THe start and end points for the cross section
    cross_start = CoordPair(lat=lat_start, lon=lon)
    cross_end = CoordPair(lat=lat_end, lon=lon)

    # Get the terrain heights along the cross section line
    ter_line_d03 = interpline(ter_d03, wrfin=wrf_file_d03, start_point=cross_start,
                            end_point=cross_end)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d03 = vertcross(Z_d03, ht_d03, wrfin=wrf_file_d03,
                        start_point=cross_start,
                        end_point=cross_end,
                        latlon=True, meta=True)



    #################################################################################
    #############################   Domain 4 #######################################
    #################################################################################


    # Filter to include only coordinate pairs with longitude greater than left_lon
    # This line of code ensures that you wont be taking a cross section with stuff from outside of the domain
    filtered_coord_pairs = [cp for cp in z_cross_d03.xy_loc.values if ((cp.lat > min_lat) & (cp.lat < max_lat))]

    # Find the coordinate pair with the longitude closest to left_lon among the filtered pairs
    # Essentially here, you're finding the coordinate pair from the d03 cross section that is closest to the boundary of domain 4
    closest_coord_pair_min = min(filtered_coord_pairs, key=lambda cp: abs(cp.lat - min_lat))
    closest_coord_pair_max = min(z_cross_d03.xy_loc.values, key=lambda cp: abs(cp.lat - max_lat)) # Same for right lon


    # Not totally sure why steps 3 and 4 need to be done, but it was givent to me by CHATGPT. 
    # Basically, the closest one to the boundary didnt work, so you need to use one more inwards.
    # Step 3: Find the index of the closest coordinate pair in the original list
    original_list = list(z_cross_d03.xy_loc.values)  # Convert to list if not already a list
    index_of_closest = original_list.index(closest_coord_pair_min)

    # Step 4: Get the next coordinate pair in the list
    if index_of_closest < len(original_list) - 1:  # Ensure it's not the last element
        next_coord_pair = original_list[index_of_closest + 1]
    else:
        next_coord_pair = None  # Handle the case where there is no next pair

    # Coordinates for the start and end of the cross section for d04. You are doing this based on the part of d03 that is within d04
    cross_start_for_d04 = next_coord_pair
    cross_end_for_d04 = CoordPair(lon=closest_coord_pair_max.lon, lat=closest_coord_pair_max.lat)

   
    # Terrain for d04
    # Get the terrain heights along the cross section line
    ter_line_d04 = interpline(ter_d04, wrfin=wrf_file_d04, start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04)
    
    # Compute the vertical cross-section interpolation.  Also, include the
    # lat/lon points along the cross-section in the metadata by setting latlon
    # to True.
    z_cross_d04 = vertcross(Z_d04, ht_d04, wrfin=wrf_file_d04,
                        start_point=cross_start_for_d04,
                            end_point=cross_end_for_d04,
                        latlon=True, meta=True)

    ############################################################################################
    
    
    # Create the figure
    fig, (ax ) = plt.subplots(1,1, figsize=(8,6), facecolor = 'white', sharey=True)

    dbz_map = plt.get_cmap('jet').copy()
    dbz_map.set_under('none')

    # Setting the background color of the plot
    # using set_facecolor() method
    ax.set_facecolor("gainsboro")


    ####################################### d03 plot ##########################################
    #dbz_levels = np.arange(0, 31., 1.0)
    xs_d03 = np.arange(0, z_cross_d03.shape[-1], 1)
    ys_d03 = to_np(z_cross_d03.coords["vertical"])

    # Fill in the mountain area
    ht_fill = ax.fill_between(xs_d03, 0, to_np(ter_line_d03),
                                    facecolor="saddlebrown", zorder = 19)


    # Do all the ticks and limits before you do the d04 plot
    #Set the x-ticks to use latitude and longitude labels
    coord_pairs = to_np(z_cross_d03.coords["xy_loc"])
    x_ticks = np.arange(coord_pairs.shape[0])
    x_labels = [pair.latlon_str() for pair in to_np(coord_pairs)]

    # Set the desired number of x ticks below
    num_ticks = 5
    thin = int((len(x_ticks) / num_ticks) + .5)
    ax.set_xticks(x_ticks[::thin])
    ax.set_xticklabels(x_labels[::thin], rotation=45, fontsize=8)

    # Set the x-axis and  y-axis labels
    ax.set_xlabel("Latitude, Longitude", fontsize=12)
    ax.set_ylabel("Height (m)", fontsize=12)

    ax.set_ylim(800,5000)
    ax.set_xlim(xs_d03[0], xs_d03[-1])


    ####################################### d04 plot ##########################################
    # Here you are extracting the x coordinates in domain 3 plot of the start and end of the cross section for domain 4
    start_y_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_start_for_d04.lat, longitude=cross_start_for_d04.lon).sel(x_y = 'y').values
    end_y_from_d04 = wrf.ll_to_xy(wrf_file_d03, latitude=cross_end_for_d04.lat, longitude=cross_end_for_d04.lon).sel(x_y = 'y').values


    # Make an array of x coordinates and y coordinates for plotting d03
    # Not sure why you need to subtract the first x from d03 cross, but it works
    xs_d04 = np.linspace(start_y_from_d04-z_cross_d03.xy_loc.values[0].y, end_y_from_d04-z_cross_d03.xy_loc.values[0].y,z_cross_d04.shape[1])
    # For the legs where the end of the leg is at the same spot as d03, just use the last point from d03
    #xs_d04 = np.linspace(start_y_from_d04-z_cross_d03.xy_loc.values[0].y, ys_d03[-1],z_cross_d04.shape[1])
    ys_d04 = to_np(z_cross_d04.coords["vertical"])

    # D04 terrain line
    ht_fill = ax.plot(xs_d04, to_np(ter_line_d04),color = 'black', zorder = 20, label = 'd04 terrain')

    # Add legend
    ax.legend(loc = 'upper left')
    
    # Add a title
    ax.set_title(f"Terrain cross section along {round(lon, 2)}°", fontsize = 10, loc = 'center',)
    ax.set_title(f'WRF{path} Run {run_number}', fontsize = 10, loc = 'left')
    
    #######################################################################
    ################### end terrain plot ##################################
    #######################################################################
    
    # Now create an inset axis for the map plot using plt.axes
    inset_ax = fig.add_axes([0.61, 0.57, 0.3, 0.3], projection=crs.PlateCarree())
    
    # Create the map in the inset
    inset_ax.contourf(ter_d03.XLONG, ter_d03.XLAT, ter_d03.values, transform=crs.PlateCarree(), cmap='terrain', levels=60)

    # Plot the cross-section line on the inset map
    inset_ax.plot([lon, lon], [lat_start, lat_end], color='red', marker='.', linewidth=2, markersize=14, transform=crs.PlateCarree(),
                 zorder = 2001)
    
    # d04 box
    inset_ax.add_patch(matplotlib.patches.Rectangle((left_lon, min_lat), right_lon-left_lon, max_lat-min_lat,
             fill=None, lw=2, edgecolor='black', zorder=2000,transform = crs.PlateCarree(), alpha = 0.5))

    plt.show()


# Define the slider using ipywidgets
lon_slider = widgets.FloatSlider(value=-112, min=left_lon+0.01, max=right_lon-0.01, step=0.01, description='Longitude:')

# Use interact to connect the slider to the plot function
interact(plot_cross_section, lon=lon_slider)

interactive(children=(FloatSlider(value=-111.8308203125, description='Longitude:', max=-111.39873291015626, mi…

<function __main__.plot_cross_section(lon)>